# RAG 單次問答系統

本範例展示：
1. **第1個儲存格**：載入向量資料庫並進行相似度檢索
2. **第2個儲存格**：整合 LLM 建立完整的 RAG 問答 Chain

學習目標：理解如何將向量檢索與 LLM 結合，建立完整的問答系統

In [ ]:
# 第1個儲存格：載入向量資料庫並進行相似度檢索

import os
from dotenv import load_dotenv
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings

# 從 .env 載入環境變數
load_dotenv()

# 定義持久化目錄
current_dir = os.path.dirname(os.path.abspath("__file__"))
persistent_directory = os.path.join(
    current_dir, "db", "chroma_db_with_metadata_chinese_nb")

# 定義嵌入模型
embeddings = HuggingFaceEmbeddings(model_name="jinaai/jina-embeddings-v2-base-zh")

# 使用嵌入函數載入現有的向量存儲
db = Chroma(persist_directory=persistent_directory,
            embedding_function=embeddings)

# 定義使用者的問題
query = "洗衣機如何清潔保養？"

# 根據查詢檢索相關文件
# search_type="similarity": 使用相似度搜尋
# k=1: 只返回最相關的 1 個文件
retriever = db.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 1},
)
relevant_docs = retriever.invoke(query)

# 顯示相關結果及元數據
print("\n--- 相關文件 ---")
for i, doc in enumerate(relevant_docs, 1):
    print(f"文件 {i}:\n{doc.page_content}\n")
    if doc.metadata:
        print(f"來源: {doc.metadata.get('source', 'Unknown')}\n")

In [ ]:
# 第2個儲存格：整合 LLM 建立完整的 RAG 問答 Chain

from langchain_core.messages import HumanMessage, SystemMessage
from langchain_ollama import ChatOllama

# 合併查詢和相關文件內容
# 這是 RAG 的核心：將檢索到的文件作為上下文提供給 LLM
combined_input = (
    "以下是一些可能有助於回答問題的文件："
    + query
    + "\n\n相關文件：\n"
    + "\n\n".join([doc.page_content for doc in relevant_docs])
    + "\n\n請僅根據提供的文件提供答案。如果在文件中找不到答案，請回覆「我不確定」。"
)

print("\n--- 合併後的輸入 ---")
print(combined_input)

# 建立 ChatOllama 模型（使用本地 LLM）
model = ChatOllama(model="llama3.2")

# 定義模型的訊息
# SystemMessage: 設定 AI 的角色和行為
# HumanMessage: 使用者的問題加上檢索到的上下文
messages = [
    SystemMessage(content="你是一個有幫助的助手。"),
    HumanMessage(content=combined_input),
]

# 使用合併的輸入調用模型
result = model.invoke(messages)

# 顯示生成的回應
print("\n--- 生成的回應 ---")
print("僅內容：")
print(result.content)

print("\n--- RAG 流程說明 ---")
print("1. 使用者提問")
print("2. 向量資料庫檢索相關文件")
print("3. 將文件作為上下文提供給 LLM")
print("4. LLM 基於上下文生成答案")